In [ ]:
!pip install openai langchain pandas openpyxl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [ ]:
# 1. Read the .xlsx data
df = pd.read_excel('Patient_Visit_Summaries.xlsx')

In [ ]:
df['Patient_ID'] = df['Patient_ID'].astype(str)

In [ ]:
!pip install openai

In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.42.0
    Uninstalling openai-1.42.0:
      Successfully uninstalled openai-1.42.0


In [ ]:
import pandas as pd
import openai

# Configuration
xlsx_file_path = 'Patient_Visit_Summaries.xlsx'  # Path to your .xlsx file
openai.api_key = 'YOUR_API_KEY'  # Replace with your OpenAI API key

# 1. Read the .xlsx data
df = pd.read_excel(xlsx_file_path)

# Ensure Patient_ID is treated as a string
df['Patient_ID'] = df['Patient_ID'].astype(str)

# 2. Define a function to filter and combine summaries
def combine_summaries(patient_id: str, patient_name: str = None) -> str:
    # Filter by Patient_ID
    filtered_df = df[df['Patient_ID'] == patient_id]

    # Optionally filter by Patient_Name
    if patient_name:
        filtered_df = filtered_df[filtered_df['Patient_Name'].str.contains(patient_name, case=False, na=False)]

    # Combine all summaries
    summaries = filtered_df['Summary'].tolist()
    combined_summary = ' '.join(summaries)

    return combined_summary

# 3. Generate a refined summary using the OpenAI API
def generate_answer(patient_id: str, patient_name: str = None) -> str:
    combined_summary = combine_summaries(patient_id, patient_name)

    if not combined_summary:
        return "No records found."

    # Construct a properly formatted prompt
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": (f"Here are the visit summaries for patient ID {patient_id} (and optionally patient name {patient_name}):\n\n"
                                      f"{combined_summary}\n\n"
                                      "Please provide a detailed summary and all data on visit basis (also include tests and measurements) based on everything given and don't exclude any information.")}
    ]

    try:
        # Make a request to the OpenAI API using GPT-4 with chat completions
        response = openai.ChatCompletion.create(
            model="gpt-4",  # Using GPT-4
            messages=messages,
            max_tokens=5000,  # Adjust max_tokens based on your needs
            temperature=0.5,  # Adjust temperature for more creativity
            top_p=0.9
        )
        return response.choices[0].message['content'].strip()
    except Exception as e:
        return f"An error occurred: {e}"

# 4. Example query
patient_id = "BLKH.1159426"  # Example Patient_ID
#patient_name = "John Doe"  # Optional Patient_Name
response = generate_answer(patient_id)
print(response)


Based on the provided information, here are the detailed summaries for each patient visit:

**Patient Visit Summary 1**
- **Visit ID:** 150698
- **Visit Type:** Inpatient (IP)
- **Care Site:** BLK Hospital
- **Doctor ID:** 37668
- **Start Date and Time:** March 27, 2024, at 21:05:34
- **Diagnosis:** Chronic Viral Hepatitis B with Delta-Agent (recorded on April 9, 2024, at 16:19:31)
- **Vital Signs:** 
    - April 15, 2024: Pulse - 62 bpm, Temperature - 98.6°F, Respiration Rate - 15 breaths per minute, Blood Pressure - 113/61 mmHg, Pulse Oximetry - 100%, Pain Level - 1
    - April 28, 2024: Pulse - 77 bpm, Temperature - 97.6°F, Respiration Rate - 21 breaths per minute, Blood Pressure - 151/61 mmHg, Pulse Oximetry - 100%, Pain Level - 1
- **Lab and Medication Information:** No lab results or medication details provided
- **Notes:** No additional comments or observations recorded

**Patient Visit Summary 2**
- **Visit ID:** 132591
- **Date and Time of Visit:** November 12, 2023, at 12:18 